# Generate a GML file to import to DHIS2
This notebook takes care of creating a GML file to import to DHIS2 from a GML file. The GML files are generated from a shape file we got from geoconnect. To generate the file we used **ogr2ogr** as described [here](https://docs.dhis2.org/2.22/en/user/html/ch18s02.html). To get the UIDs of the organizations in DHIS2 we exported the organization unit metadata using DHIS2's import export module. In this notebook, this file is called *ntddb_orgs.json*.  


Running through this entire notebook will generate 3 GML files, one for each administrative level that we have geographic information for These files are called admin1-ethiopia.xml for region level boundries, admin2-ethiopia.xml for district level boundries, and admin3-ethiopia.xml for woreada level boundries

In [18]:
# use this to quickly look up an ORG ID, we'll use this quite a bit
def admin_org_search(name, orgs):
    return [element['id'] for element in orgs if element['name'].strip() == name]


# Generate Region level GML
where admin1s is used to generate the admin1 (region) GML file and admin1_ids is used for building out the admin file (district)

In [36]:
import json


parent = "tfrwwy49VMx"
#org_file_name = "ntddb_orgs.json" # https://ntddb.callawaywilson.com/
org_file_name = "ntd2-orgs.json" 

#parent = "iuGjpnxnFbI" # https://ntddb.callawaywilson.com/

data = json.load(open(org_file_name))
admin1s = []
admin1_ids = []
for org in data["organisationUnits"]:
    if "parent" in org and org["parent"]["id"] == parent:
        admin1 = {"name": org["name"], "id": org["id"]}
        admin1s.append(admin1)
        admin1_ids.append(org["id"])

### Get the for Ethiopia regions in the GML file

In [16]:
import xml.etree.ElementTree as ET
tree1 = ET.parse('/Users/apreston/Ethiopia/06092017_update/admin1.gml')
root1 = tree1.getroot()

In [41]:
admin1s

[{'id': 'ORG00000001', 'name': 'Addis Ababa'},
 {'id': 'ORG00000013', 'name': 'Afar'},
 {'id': 'ORG00000053', 'name': 'Amhara'},
 {'id': 'ORG00000218', 'name': 'Beneshangul Gumuz'},
 {'id': 'ORG00000256', 'name': 'Dire Dawa'},
 {'id': 'ORG00000267', 'name': 'Gambella'},
 {'id': 'ORG00000299', 'name': 'Harare'},
 {'id': 'ORG00000310', 'name': 'Oromia'},
 {'id': 'ORG00000667', 'name': 'SNNPR'},
 {'id': 'ORG00000842', 'name': 'Somali'},
 {'id': 'ORG00000933', 'name': 'Tigray'}]

### Build out list of admin1 elements (with special UID attribute)

In [30]:

ethiopia_admin1_orgs = []

for orgxml in root1.iter('{http://www.opengis.net/gml}featureMember'):
    

    # filter on country = Ethopia
    if (orgxml[0].find("{http://ogr.maptools.org/}ADMIN0").text == "Ethiopia"):
        
        org_id = admin_org_search(orgxml[0].find("{http://ogr.maptools.org/}ADMIN1").text, admin1s)[0]
        ET.SubElement(orgxml[0], "rti:UID").text = org_id
        
        ethiopia_admin1_orgs.append(orgxml)
         

### Write out XML

In [31]:
import xml.etree.cElementTree as ET

ET.register_namespace('ogr', "http://ogr.maptools.org/")
ET.register_namespace('gml', "http://www.opengis.net/gml")
ET.register_namespace('rti', "https://www.rti.org/")


root = ET.Element("ogr:FeatureCollection")
root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
root.set("xsi:schemaLocation", "http://ogr.maptools.org/ Admin1.xsd")
root.set("xmlns:rti", "https://www.rti.org/")

# go through and add feature members
for element in ethiopia_admin1_orgs:
    root.append(element)

tree = ET.ElementTree(root)

tree.write("admin1-ethiopia.xml", encoding='utf-8', xml_declaration=True)

In [43]:
admin1_ids

['ORG00000001',
 'ORG00000013',
 'ORG00000053',
 'ORG00000218',
 'ORG00000256',
 'ORG00000267',
 'ORG00000299',
 'ORG00000310',
 'ORG00000667',
 'ORG00000842',
 'ORG00000933']

#  Genderate district level GML
where admin2s is used to generate the admin2 (region) GML file and admin2_ids is used for building out the admin file (woreada)

In [46]:
org_file_name

'ntd2-orgs.json'

In [61]:
import json


data = json.load(open(org_file_name))
admin2s = []
admin2_ids = []

for org in data["organisationUnits"]:
    
    if "parent" in org and org["parent"]["id"] in admin1_ids:

        admin2 = {"name": org["name"], "id": org["id"]}
        # print(admin2)
        admin2s.append(admin2)
        admin2_ids.append(org["id"])

In [60]:
#admin2s

### Get the for Ethiopia regions in the GML file

In [5]:
import xml.etree.ElementTree as ET
tree2 = ET.parse('/Users/apreston/Ethiopia/06092017_update/Admin2.gml')
root2 = tree2.getroot()

### Build out list of admin2 elements (with special UID attribute)

In [62]:
ethiopia_admin2_orgs = []

for orgxml in root2.iter('{http://www.opengis.net/gml}featureMember'):
    
    # filter on country = Ethopia
    if (orgxml[0].find("{http://ogr.maptools.org/}ADMIN0").text == "Ethiopia"):
        org_id = admin_org_search(orgxml[0].find("{http://ogr.maptools.org/}ADMIN2").text, admin2s)
        org_name = orgxml[0].find("{http://ogr.maptools.org/}ADMIN2").text
        region_name = orgxml[0].find("{http://ogr.maptools.org/}ADMIN1").text
        
        if (len(org_id) == 0):
            print(org_name + " is not found as an admin2 in DHIS2 in region " 
                  + region_name + ". Skipping.")
        else:
            ET.SubElement(orgxml[0], "rti:UID").text = org_id[0]
            ethiopia_admin2_orgs.append(orgxml)
                 

## Write out XML

In [63]:
import xml.etree.cElementTree as ET

ET.register_namespace('ogr', "http://ogr.maptools.org/")
ET.register_namespace('gml', "http://www.opengis.net/gml")
ET.register_namespace('rti', "https://www.rti.org/")

root = ET.Element("ogr:FeatureCollection")
root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
root.set("xsi:schemaLocation", "http://ogr.maptools.org/ Admin1.xsd")
root.set("xmlns:rti", "https://www.rti.org/")

# go through and add feature members
for element in ethiopia_admin2_orgs:
    root.append(element)

tree = ET.ElementTree(root)

tree.write("admin2-ethiopia.xml", encoding='utf-8', xml_declaration=True)

# Generate woreada level GML
where *admin3s* is used to generate the admin3 (woreada) GML file  

In [64]:
import json

data = json.load(open(org_file_name))
admin3s = []

for org in data["organisationUnits"]:
    if "parent" in org and org["parent"]["id"] in admin2_ids: # make sure a parent is one of the districts
        admin3 = {"name": org["name"], "id": org["id"]}
        admin3s.append(admin3)

In [66]:
#admin3s

### Get the for Ethiopia woreada in the GML file

In [67]:
import xml.etree.ElementTree as ET
tree3 = ET.parse('/Users/apreston/Ethiopia/06092017_update/Admin3.gml')
root3 = tree3.getroot()

### Build out list of admin3 elements (with special UID attribute)

In [68]:
ethiopia_admin3_orgs = []

for orgxml in root3.iter('{http://www.opengis.net/gml}featureMember'):
    
    if (orgxml[0].find("{http://ogr.maptools.org/}ADMIN0").text == "Ethiopia"):

        org_name = orgxml[0].find("{http://ogr.maptools.org/}ADMIN3").text
        parent_name = orgxml[0].find("{http://ogr.maptools.org/}ADMIN2").text
        region_name = orgxml[0].find("{http://ogr.maptools.org/}ADMIN1").text
        
        org_id = admin_org_search(orgxml[0].find("{http://ogr.maptools.org/}ADMIN3").text, admin3s)
        
        if (len(org_id) == 0):
            print(org_name + " is not found as an admin3 in DHIS2 in district/region " 
                  + parent_name + "/"
                  + region_name + ". Skipping.")
        else:
            ET.SubElement(orgxml[0], "rti:UID").text = org_id[0]
            ethiopia_admin3_orgs.append(orgxml)


Harare is not found as an admin3 in DHIS2 in district/region Harare/Harare. Skipping.
Halaba is not found as an admin3 in DHIS2 in district/region Special/SNNPR. Skipping.
Qerca is not found as an admin3 in DHIS2 in district/region Guji/Oromia. Skipping.
Jijiga Rural is not found as an admin3 in DHIS2 in district/region Faafame/Somali. Skipping.


In [69]:
import xml.etree.cElementTree as ET

ET.register_namespace('ogr', "http://ogr.maptools.org/")
ET.register_namespace('gml', "http://www.opengis.net/gml")
ET.register_namespace('rti', "https://www.rti.org/")

root = ET.Element("ogr:FeatureCollection")
root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
root.set("xsi:schemaLocation", "http://ogr.maptools.org/ Admin1.xsd")
root.set("xmlns:rti", "https://www.rti.org/")

# go through and add feature members
for element in ethiopia_admin3_orgs:
    root.append(element)

tree = ET.ElementTree(root)

tree.write("admin3-ethiopia.xml", encoding='utf-8', xml_declaration=True)

In [336]:
# testing
org_id = admin_org_search("Harare", admin3s)
org_id

[]

In [338]:
#admin3s